## Web Scraping

We create this file for testing our web scrapping code because this notebook is easy to debug.<br/>
We will explain our code in a comment on the code. <br/> 
reference link: https://weather.com/weather/tenday/l/Boston+MA?canonicalCityId=6320cadd3d539b434b5a45c094becf3edbe8ea88958185a2287a801115c9ae30

In [1]:
import requests
from bs4 import BeautifulSoup
page=requests.get("https://weather.com/weather/tenday/l/Boston+MA?canonicalCityId=6320cadd3d539b434b5a45c094becf3edbe8ea88958185a2287a801115c9ae30")
# In the above code, we imported the request library and stored data from URL into the response object page
content=page.content
soup=BeautifulSoup(content,"html.parser")
# In the above code BeautifulSoup is imported from bs4 and parsed html page content.
# So now soup contains the parsed content of the web page.
l=[]
all=soup.find("div",{"class":"locations-title ten-day-page-title"}).find("h1").text
 
# We can use find_all method to pick up all the html elements of tag passed in as argument and it’s descendants. 
# Whereas, find method will search for the elements of the tag passed, div is passed here so, we shall get all the results of div tag from the page content.
table=soup.find_all("table",{"class":"twc-table"})
for items in table:
    for i in range(len(items.find_all("tr"))-1):
        d = {}
        try:
            d["day"]=items.find_all("span",{"class":"date-time"})[i].text
            d["date"]=items.find_all("span",{"class":"day-detail"})[i].text
            d["desc"]=items.find_all("td",{"class":"description"})[i].text
            d["temp"]=items.find_all("td",{"class":"temp"})[i].text
            d["precip"]=items.find_all("td",{"class":"precip"})[i].text
            d["wind"]=items.find_all("td",{"class":"wind"})[i].text
            d["humidity"]=items.find_all("td",{"class":"humidity"})[i].text
        except:
            d["day"]="None"
            d["date"]="None"
            d["desc"]="None"
            d["temp"]="None"
            d["precip"]="None"
            d["wind"]="None"
            d["humidity"]="None"
            
        l.append(d) #l.append(d) ,we are appending all the data to the l of type list. 
l

[{'day': 'Tonight',
  'date': 'DEC 16',
  'desc': 'Snow Late',
  'temp': '--29°',
  'precip': '80%',
  'wind': 'NW 6 mph ',
  'humidity': '56%'},
 {'day': 'Tue',
  'date': 'DEC 17',
  'desc': 'Wintry Mix to Rain',
  'temp': '38°31°',
  'precip': '100%',
  'wind': 'NE 14 mph ',
  'humidity': '85%'},
 {'day': 'Wed',
  'date': 'DEC 18',
  'desc': 'Partly Cloudy',
  'temp': '38°14°',
  'precip': '10%',
  'wind': 'W 16 mph ',
  'humidity': '56%'},
 {'day': 'Thu',
  'date': 'DEC 19',
  'desc': 'Sunny',
  'temp': '24°16°',
  'precip': '0%',
  'wind': 'WNW 19 mph ',
  'humidity': '43%'},
 {'day': 'Fri',
  'date': 'DEC 20',
  'desc': 'Sunny',
  'temp': '28°17°',
  'precip': '0%',
  'wind': 'NW 13 mph ',
  'humidity': '53%'},
 {'day': 'Sat',
  'date': 'DEC 21',
  'desc': 'Partly Cloudy',
  'temp': '35°28°',
  'precip': '10%',
  'wind': 'WNW 7 mph ',
  'humidity': '57%'},
 {'day': 'Sun',
  'date': 'DEC 22',
  'desc': 'Partly Cloudy',
  'temp': '43°31°',
  'precip': '20%',
  'wind': 'SW 9 mph ',
 